In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
import csv
import urllib.request
%matplotlib inline

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
# Generate csv file from xeno-canto API

# specify url
web_url = "https://xeno-canto.org/api/2/recordings?query=sp&field=species/all?area=africa"
html = urlopen(web_url)

In [ ]:
soup = BeautifulSoup(html, 'html.parser')
type(soup)

In [ ]:
# Print out the text
text = soup.get_text()
print(text)

In [ ]:
# convert to dataframe
df = pd.DataFrame(parsed_json['recordings'])
df.head()


In [ ]:
# generate csv file from the json data
df.columns

In [ ]:
# convert data frame into csv file
df.to_csv('bird_song_dataset.csv', index=False)

In [ ]:
# extract bird songs from the website
url = "https://xeno-canto.org/api/2/recordings?query=sp&field=species/all?area=africa"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    if "recordings" in data:
        species_list = list({recording["sp"] for recording in data["recordings"]})

        # Specify the file path for the CSV file
        csv_file_path = "species_list.csv"

        # Write the species_list to the CSV file
        with open(csv_file_path, "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["Species"])  # Write header row
            writer.writerows([[species] for species in species_list])  # Write species names row by row

        print("Data saved to CSV file successfully.")
else:
    print("Error: Unable to fetch data from the API.")

In [ ]:
df = pd.read_csv('species_list.csv')
df.head()

In [ ]:
import os
import requests
import urllib.request

# Xeno-Canto API endpoint for bird songs
api_url = "https://xeno-canto.org/api/2/recordings?query=sp&field=species/all?area=africa"

def download_audio(url, file_path):
    """Download audio file from the given URL and save it to the specified file path."""
    try:
        urllib.request.urlretrieve(url, file_path)
        print(f"Audio file saved: {file_path}")
    except Exception as e:
        print(f"Error downloading audio: {e}")

def extract
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()

            # Create a main folder to store species subdirectories
            main_folder = "bird_species_audio"
            if not os.path.exists(main_folder):
                os.makedirs(main_folder)

            for recording in data["recordings"]:
                # Extract relevant information from the recording
                species = recording["en"].replace("/", "-")  # Replace '/' with '-' to avoid issues with subdirectories
                audio_url = recording["file"]
                folder_name = os.path.join(main_folder, species)

                # Create a subdirectory for the species (if it doesn't exist)
                if not os.path.exists(folder_name):
                    os.makedirs(folder_name)

                # Generate a unique filename using the recording ID
                recording_id = recording["id"]
                file_name = f"{species}_{recording_id}.mp3"
                file_path = os.path.join(folder_name, file_name)  # Save the audio with the unique filename in the species folder

                # Download the audio file
                download_audio(audio_url, file_path)

        else:
            print(f"Error fetching data from Xeno-Canto API. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error extracting bird songs: {e}")

if __name__ == "__main__":
    extract_bird_songs(api_url)
